In [10]:
from keras.models import load_model
import keras.backend as K
from keras.layers import *
from keras import Model
import numpy as np
import tensorflow as tf
from util import *
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
triplets = np.load('obj/triplets.npy')

In [45]:
def gram(x):
    m, n = map(int, x.shape[2:])
    G = gram_matrix(x)
    return G / (4 * m**2 * n**2)

def get_style_model():
    model = load_model('obj/darknet_avgpooling.h5')
    for layer in model.layers[:-12]:
        layer.trainable = False
    x = model.get_layer('average_pooling2d_6').output
    x = GlobalAveragePooling2D()(x)
    return Model(model.input, x, name='style_model')

In [46]:
shape = (256, 256, 3)

def style_trainer(triplets, model, epochs):
    input_a = Input(shape)
    input_p = Input(shape)
    input_n = Input(shape)

    emb_a = model(input_a)
    emb_p = model(input_p)
    emb_n = model(input_n)

    dist_p = Lambda(euclSq)([emb_a, emb_p])
    dist_n = Lambda(euclSq)([emb_a, emb_n])
    loss = Lambda(triplet_loss)([dist_p, dist_n])
    triple_model = Model([input_a, input_p, input_n], loss)

    triple_model.compile(optimizer='adam', loss='mean_absolute_error')
    alpha = np.zeros(triplets.shape[0])
    triple_model.fit([triplets[:, 0], triplets[:, 1], triplets[:, 2]], alpha, epochs=epochs, batch_size=32)

In [47]:
model = get_style_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 16)      432       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 16)      64        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256, 256, 16)      0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 32)      4608      
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 32)      128       
__________

/home/henrik/miniconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [48]:
style_trainer(triplets, model, 40)

Epoch 1/40
800/800 [==============================] - 5s 7ms/step - loss: 1.9793
Epoch 2/40
800/800 [==============================] - 3s 4ms/step - loss: 0.7315
Epoch 3/40
800/800 [==============================] - 3s 4ms/step - loss: 0.4998
Epoch 4/40
800/800 [==============================] - 3s 4ms/step - loss: 0.3891
Epoch 5/40
800/800 [==============================] - 3s 4ms/step - loss: 0.3391
Epoch 6/40
800/800 [==============================] - 3s 4ms/step - loss: 0.3190
Epoch 7/40
800/800 [==============================] - 3s 4ms/step - loss: 0.2864
Epoch 8/40
800/800 [==============================] - 3s 4ms/step - loss: 0.2846
Epoch 9/40
800/800 [==============================] - 3s 4ms/step - loss: 0.2795
Epoch 10/40
800/800 [==============================] - 3s 4ms/step - loss: 0.2789
Epoch 11/40
800/800 [==============================] - 3s 4ms/step - loss: 0.2800
Epoch 12/40
800/800 [==============================] - 3s 4ms/step - loss: 0.2888
Epoch 13/40
800/800 [====

In [49]:
model.save('obj/style_model.h5')